 # Récupération Donnée API Github

Chargement de la clé github 

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")

CLE_API_GITHUB = os.getenv('CLE_API_GITHUB')

In [2]:
import requests
import random
import base64
import time

HEADERS = {"Authorization": f"token {CLE_API_GITHUB}"}

def get_repos():
    # Recherche générale avec le mot "a" (ou autre lettre)
    url = "https://api.github.com/search/repositories"
    params = {
        "q": "a", # recherche avec la lettre "a"
        "order": "desc",
        "per_page": 5, 
        "page": random.randint(1, 100), # page random entre 1 et 100
        }  
    response = requests.get(url, headers=HEADERS, params=params)
    data_repos = []
    
    if response.status_code == 200:
        data = response.json()
        repos = data.get("items", [])
        if repos:
            for repo in repos:
                if(fetch_and_check_readme(repo)):
                    data_repos.append({
                        "name": repo["name"],
                        "html_url": repo["html_url"],
                        "description": repo["description"],
                        "readme" : repo["readme_content"]
                    })
            return data_repos
        else:
            return "Aucun dépôt trouvé."
    else:
        if response.status_code == 403:
            time.sleep(10)
            return get_repos()
        else :
            return f"Erreur: {response.status_code}, {response.text}"


def fetch_and_check_readme(repo):
    readme_url = repo["url"] + "/readme"
    response = requests.get(readme_url, headers=HEADERS)
    readme_is_present = False
    if response.status_code == 200:
        data = response.json()
        if "content" in data:
            # Décoder le contenu encodé en base64
            content = base64.b64decode(data["content"]).decode("utf-8").strip()
            if content:  # Vérifier que le README n'est pas vide
                repo["readme_content"] = content
                readme_is_present = True
    return readme_is_present


In [10]:
import schedule
import time
import json
from spark_kafka import produce_messages

def find_and_send_repos():
    repo = get_repos()
    print(repo)
    for r in repo:
        print(f"Envoi du repo {r['name']} à Kafka")
        #produce_messages(json.dumps(repo, indent=4), "add_to_es")
"""
if __name__ == "__main__":
    schedule.every(2).minutes.do(find_and_send_repos)
    while True:
        schedule.run_pending()
        time.sleep(10)
"""

find_and_send_repos()

Erreur: 403, {
  "documentation_url": "https://docs.github.com/free-pro-team@latest/rest/overview/rate-limits-for-the-rest-api#about-secondary-rate-limits",
  "message": "You have exceeded a secondary rate limit. Please wait a few minutes before you try again. If you reach out to GitHub Support for help, please include the request ID C6E1:32A54B:FF4958:10528B6:678BEB4E."
}


TypeError: string indices must be integers